<a href="https://colab.research.google.com/github/MinJunJA/SQL_study/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#cnn
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# 데이터셋 디렉토리 경로 설정
data_dir = '/content/drive/MyDrive/babycrying/sample'

# 클래스 레이블 설정 (클래스별 디렉토리 이름으로 가정)
class_labels = os.listdir(data_dir)

# MFCC 계수 개수 설정
n_mfcc = 13

# 데이터셋과 레이블을 저장할 리스트
X = []
labels = []

# 데이터셋 불러오기
for label_idx, label in enumerate(class_labels):
    class_dir = os.path.join(data_dir, label)
    for filename in os.listdir(class_dir):
        file_path = os.path.join(class_dir, filename)
        y, sr = librosa.load(file_path, sr=None)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        X.append(mfccs.T)
        labels.append(label)

# NumPy 배열로 변환
X = np.array(X)

# 레이블을 정수형으로 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
num_classes = len(class_labels)
labels = to_categorical(labels, num_classes=num_classes)

# 최대 길이에 맞춰 MFCC 벡터를 패딩
max_length = max(len(mfccs) for mfccs in X)
X = np.array([np.pad(mfccs, ((0, max_length - len(mfccs)), (0, 0)), mode='constant') for mfccs in X])

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


#모델생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same')) #padding = 'same'을 넣지 않으면 다운 샘플링을 시도하여 모델이 돌아가지 않음
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# 모델 평가
accuracy = model.evaluate(X_test, y_test)[1]
print("Accuracy:", accuracy)

<ipython-input-3-dcf97473d088>:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


Epoch 1/50
96/96 [==============================] - 14s 135ms/step - loss: 1.8379 - accuracy: 0.3239 - val_loss: 1.4016 - val_accuracy: 0.3275
Epoch 2/50
96/96 [==============================] - 15s 152ms/step - loss: 1.3856 - accuracy: 0.3646 - val_loss: 1.3442 - val_accuracy: 0.4035
Epoch 3/50
96/96 [==============================] - 15s 157ms/step - loss: 1.3811 - accuracy: 0.3825 - val_loss: 1.3223 - val_accuracy: 0.3889
Epoch 4/50
96/96 [==============================] - 13s 132ms/step - loss: 1.3846 - accuracy: 0.3858 - val_loss: 1.3424 - val_accuracy: 0.4152
Epoch 5/50
96/96 [==============================] - 12s 123ms/step - loss: 1.3627 - accuracy: 0.4167 - val_loss: 1.2878 - val_accuracy: 0.4444
Epoch 6/50
96/96 [==============================] - 12s 120ms/step - loss: 1.2986 - accuracy: 0.4308 - val_loss: 1.2517 - val_accuracy: 0.4503
Epoch 7/50
96/96 [==============================] - 13s 132ms/step - loss: 1.2876 - accuracy: 0.4542 - val_loss: 1.2308 - val_accuracy: 0.4620